In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['clean_gdelt_data']

In [2]:
# Parameters
upstream = {
    "clean_gdelt_data": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/clean_gdelt_data.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/interim/cleaned-gdelt-data.csv",
    }
}
product = {"nb": "/Users/aiujdm2/market_watch/output/notebooks/visualize_gdelt.ipynb"}


In [3]:
import pandas as pd
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
alt.themes.enable('fivethirtyeight')

ThemeRegistry.enable('fivethirtyeight')

### Load Data File to Visualize

The file path to data file is inserted automatically by Ploomber using upstream task data dictionary

In [4]:
input_file_path = str(upstream['clean_gdelt_data']['data'])

In [5]:
def read_data(input_file_path):
    print(f"reading {input_file_path}")
    data = pd.read_csv(input_file_path, index_col=0)
    return data

In [6]:
data_df = read_data(input_file_path)

reading /Users/aiujdm2/market_watch/output/data/interim/cleaned-gdelt-data.csv


In [7]:
dates = data_df.GKGRECORDID.str.extract(pat="([0-9]+)")
data_df['Date'] = pd.to_datetime(dates[0], format="%Y%m%d%H%M%S")

In [8]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 0 to 103
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   GKGRECORDID    104 non-null    object        
 1   Locations      104 non-null    object        
 2   Persons        86 non-null     object        
 3   Organizations  104 non-null    object        
 4   AvgTone        104 non-null    float64       
 5   PosScore       104 non-null    float64       
 6   NegScore       104 non-null    float64       
 7   Polarity       104 non-null    float64       
 8   Org Count      104 non-null    object        
 9   Date           104 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(5)
memory usage: 8.9+ KB


### Line chart to see any obvious trends with sentiments on Tesla related news
Note that we do not see any major shift in trend. It is however trending positive (trending to be more negative here in context of NegScore)

In [9]:
base = alt.Chart(data_df)
line_chart = base.mark_line().transform_window(
    # The field to average
    rolling_mean='mean(NegScore)',
    # The number of values before and after the current value to include.
    frame=[360, 0]
).encode(
    alt.X('Date:T'),
    alt.Y('rolling_mean:Q', )
).properties(
    width = 900
).configure(
    font='monospace'
)
line_chart

alt.Chart(...)